In [86]:
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import open3d as o3d

# 深度图的宽度和高度
depth_width = 640
depth_height = 480

# 创建一个深度图（示例，实际应该根据你的深度数据来创建）
depth_image = Image.open('image_93.png')
depth_image = np.array(depth_image)



# 相机内部参数
fx = 392.323 # 水平焦距
fy = 392.323 # 垂直焦距
cx = 319.158 # 主点的x坐标
cy = 240.853 # 主点的y坐标

min_depth = 0.6
max_depth = 10.0
DEPTH_SCALE_M = 0.001
nums = 3


# 定义LiDAR扫描参数
num_scan_points = 90  # 90度扫描，可以根据需要调整
lines = 64  # 90度扫描，可以根据需要调整
min_angle = -np.pi / 4  # 最小扫描角度
max_angle = np.pi / 4  # 最大扫描角度
# 初始化LiDAR点云数据
lidar_points = []
digess = num_scan_points / lines
v_angle = [round(np.radians(i),nums) for i in np.arange(-45,18,digess)]
# h_angle = [round(np.radians(i),nums) for i in np.arange(-45,45)]


In [87]:
np.degrees(np.arctan2(1,3))

18.43494882292201

In [88]:

# 将深度图转换为点云

for y in range(depth_image.shape[0]):
    for x in range(depth_image.shape[1]):
        # 获取深度值
        depth = depth_image[y, x] * DEPTH_SCALE_M

        # 计算对应的点的坐标（假设 Z 轴为深度，X 和 Y 轴为图像平面）
        if depth > 0:
            
            point_x = (x - cx) * depth / fx
            point_y = (y - cy) * depth / fy
            point_z = depth
            
            distance = np.sqrt(point_x**2 + point_y**2 + point_z**2)
            
            # alpha = round(np.arctan2(point_x,distance), nums)
            beta = round(np.arctan2(point_y,distance), nums)
            # alpha in h_angle and 
            if not (beta in v_angle) :
                continue

            # 将点的坐标添加到点云列表中
            lidar_points.append([point_x, point_y, point_z])

# lidar_points 现在包含了模拟的LiDAR点云数据


In [ ]:

# # 可选：添加噪声
# lidar_points = np.array(lidar_points)  # 转换为NumPy数组以便处理
# noise_stddev = 0.01  # 噪声标准差
# noise = np.random.normal(0, noise_stddev, size=lidar_points.shape)
# lidar_points += noise

# 绕 Y 轴顺时针旋转 90 度
angle_y = np.radians(90)  # 将角度转换为弧度
rotation_y = np.array([[np.cos(angle_y), 0, np.sin(angle_y)],
                    [0, 1, 0],
                    [-np.sin(angle_y), 0, np.cos(angle_y)]])
rotated_points = np.dot(lidar_points, rotation_y.T)
# 绕 X 轴逆时针旋转 90 度
angle_x = np.radians(-90)  # 将角度转换为弧度
rotation_x = np.array([[1, 0, 0],
                    [0, np.cos(angle_x), -np.sin(angle_x)],
                    [0, np.sin(angle_x), np.cos(angle_x)]])
rotated_points = np.dot(rotated_points, rotation_x.T)

nan_mask = np.isnan(rotated_points).any(axis=1)
lidar_points = rotated_points[~nan_mask]




In [ ]:


pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(lidar_points)
pcd.paint_uniform_color([1.0, 0.5, 0.0])  # 设置橙色

# pcd2 = o3d.geometry.PointCloud()
# pcd2.points = o3d.utility.Vector3dVector(unproce_point_cloud)
# pcd2.paint_uniform_color([1.0, 0.0, 0.0])  # 设置红色

lidar_point_cloud  = np.load('lidarpoint_93.npy')
lidar_point_cloud[:,2] += 0.22
pcd3 = o3d.geometry.PointCloud()
pcd3.points = o3d.utility.Vector3dVector(lidar_point_cloud)
pcd3.paint_uniform_color([0.0, 0.5, 0.3])  # 设置红色


coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0, 0, 0])

# 创建一个场景并将点云、包围框和坐标系添加到场景中
scene = o3d.visualization.Visualizer()
scene.create_window()
scene.add_geometry(pcd)        # 添加点云
# scene.add_geometry(pcd2)        # 添加点云
# scene.add_geometry(pcd3)        # 添加点云
scene.add_geometry(coord_frame)  # 添加坐标系


# 设置视角
view_control = scene.get_view_control()
view_control.set_lookat([0, 0, 0])  # 设置视点
view_control.set_up([1, 0, 0])     # 设置视角上方的方向

# 显示场景
scene.run()
scene.destroy_window()